# Shortest path LP

### Author: [Ben Rosenberg](https://benrosenberg.info)

### Imports

We begin by importing some relevant libraries. We import `ortools` to formulate and solve the LP, and we import `time` to time the entire process of supplying the constraints solving the LP.

In [14]:
from ortools.linear_solver import pywraplp as OR
import time

### Input data

Next, we define our input data. Recall that in the shortest path problem we have as an input a graph $G = (N, E)$, where $N$ is the set of nodes, and $E$ is the set of edges, each of which has a capacity $u(i,j)$ for all $(i,j)\in E$.

The input data below corresponds to a small example graph, seen below, in which the numbers inside the nodes are arbitrary indices, and the number corresponding to each edge denotes the capacity of that edge:

![](max_flow_graph.png)

We will create and solve an LP to determine the shortest path from node 0 to node 5.

In [15]:
nodes = range(6)

edges = [(0,1), (0,3), (1,2), (1,3), (2,3), (2,5), (3,4), (4,5)]

# cost[i,j] is the length of edge (i,j)
cost = {
    (0,1) : 5, 
    (0,3) : 3, 
    (1,2) : 3, 
    (1,3) : 1, 
    (2,3) : 2, 
    (2,5) : 6, 
    (3,4) : 4, 
    (4,5) : 3
}

### Model Definition

Now we define our model. The shortest path problem has the following constraints, using the variable $x(i,j)$ to denote the whether edge $(i,j)$ is included in the shortest path:

 - Net flow (out minus in) is zero for non-start/finish nodes
 $$\sum_{(j,i)\in E} x(j,i) - \sum_{(i,j)\in E} x(i,j) = 0 \quad \forall i\in N\backslash\{s,t\}$$
 - Net flow (out minus in) is 1 for $s$ and -1 for $t$. In this case $s = 0$ and $t = 5$, so we can insert those directly.
 $$\sum_{(0,i)\in E} x(0,i) - \sum_{(i,0)\in E} x(i,0) = 1$$
 $$\sum_{(5,i)\in E} x(5,i) - \sum_{(i,5)\in E} x(i,5) = -1$$
 - Flow on an edge can be no more than the capacity of that edge (all edges have capacity 1 in the shortest path formulation)
 $$x(i,j) \leq 1 \quad \forall (i,j)\in E$$
 - Flow on an edge can be no less than 0 units
 $$x(i,j) \geq 0 \quad \forall (i,j) \in E$$
 
And the objective is minimizing the total cost of the shortest path:

$$\min \sum_{(i,j)\in E} x(i,j) \cdot c(i,j)$$

In [16]:
start_time = time.time()

m = OR.Solver('Shortest path', OR.Solver.GLOP_LINEAR_PROGRAMMING)

# add variable x (this includes the 0 <= x <= 1 constraints)
x = {}
for (i,j) in edges:
    x[i,j] = m.NumVar(0, 1, 'x[{},{}]'.format(
        i,j
    ))

# add constraint flow in/out for non-source/sink nodes
for i in nodes:
    if i not in (0,5):
        m.Add(sum(x[j,i] for (j,y) in edges if y == i) - 
              sum(x[i,j] for (y,j) in edges if y == i)
              == 0)

# add constraint flow in/out for source/sink nodes
# source node
m.Add(sum(x[0,i] for (j,i) in edges if j == 0) -
      sum(x[i,0] for (i,j) in edges if j == 0)
      == 1)
# sink node
m.Add(sum(x[5,i] for (j,i) in edges if j == 5) - 
      sum(x[i,5] for (i,j) in edges if j == 5)
      == -1)

# set objective
m.Minimize(sum(x[i,j] * cost[i,j] for (i,j) in edges))

m.Solve()

end_time = time.time()

diff = time.gmtime(end_time - start_time)
print('\n[Total time used: {} minutes, {} seconds]'.format(diff.tm_min, diff.tm_sec))

print('Objective:', m.Objective().Value())


[Total time used: 0 minutes, 0 seconds]
Objective: 10.0


So we have our optimal objective. The optimal solution associated with that, in terms of flows on edges, is given below:

In [17]:
# optimal solution
for (i,j) in edges:
    print('x[{},{}] = {}'.format(
        i, j, x[i,j].solution_value()
    ))

x[0,1] = 0.0
x[0,3] = 1.0
x[1,2] = 0.0
x[1,3] = 0.0
x[2,3] = 0.0
x[2,5] = 0.0
x[3,4] = 1.0
x[4,5] = 1.0
